In [2]:
import json
import requests
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup
from pymongo import MongoClient
import telegram


# Function to send notification to telegram chat
async def send_notification(title, link):
    await bot.send_message(chat_id, f'{title}\n{link}\n')
    time.sleep(3)


# Load credentials from JSON file
with open('credentials.json') as f:
    credentials = json.load(f)

# Extract MongoDB and Telegram credentials from the credentials object
mongo_creds = credentials['mongo']
telegram_creds = credentials['telegram']

# Connect to MongoDB
client = MongoClient(mongo_creds['host'])
db = client[mongo_creds['database']]
collection = db[mongo_creds['collection']]

# Connect to Telegram bot
bot = telegram.Bot(token=telegram_creds['token'])
chat_id = telegram_creds['chat_id']

# URLs to scrape and corresponding CSS classes for titles
websites = ['https://forum.donanimarsivi.com/forumlar/Sicakfirsatlar/',
            'https://forum.donanimhaber.com/sicak-firsatlar--f193?sayfa=1',
            'https://www.r10.net/sicak-firsatlar']
classes = ["structItem-title", "kl-konu", "title"]

# Base URL for each website (used to construct the full topic URL)
firsat = ['https://forum.donanimarsivi.com', 'https://forum.donanimhaber.com', '']

# Infinite loop to continuously scrape the websites
while True:
    for i, url in enumerate(websites):

        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        topics = soup.find_all('div', {'class': classes[i]})

        for topic in topics[1:]:
            title_elem = topic.find('a', {'class': ''})
            if not title_elem:
                continue
            title = title_elem.text.strip()
            if url == 'https://forum.donanimhaber.com/sicak-firsatlar--f193?sayfa=1':
                try:
                    topic_link = firsat[i] + title_elem.get("href")
                    if any(x in title for x in ["[ANA KONU]", "ANA KONU", "[ANAKONU]", "Sabit Konu: ",
                                                "Sponsorlu İçerik:", "ad.donanim"]) or "ad.donanim" in topic_link:
                        continue
                except Exception:
                    continue
            else:
                topic_link = firsat[i] + title_elem.get("href")

            existing_doc = collection.find_one({"topic": title})

            if existing_doc is None:
                # title_real = title.replace("\\", "").strip()
                doc = {
                    "creation": current_time,
                    "topic": title,
                    "link": topic_link
                }
                collection.insert_one(doc)
                await send_notification(title, topic_link)

    time.sleep(180)
